<a href="https://colab.research.google.com/github/hfoley2013/hawku_sales/blob/main/HawkuSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
#@title ZED Run Transaction Screener
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

df = pd.read_csv('/content/hawku_sales_data/hawku_sales.csv', parse_dates=['Date'])
pd.set_option('display.max_rows', 100)

# Create dropdown filters for Bloodline, Breed, and Coat Color
bloodline_options = ['All'] + df['Bloodline'].unique().tolist()
bloodline_filter = widgets.Dropdown(options=bloodline_options, description='Bloodline Filter:')

breed_options = ['All'] + df['Breed'].unique().tolist()
breed_filter = widgets.Dropdown(options=breed_options, description='Breed Filter:')

coat_color_options = ['All'] + df['Coat Color'].unique().tolist()
coat_color_filter = widgets.Dropdown(options=coat_color_options, description='Coat Color Filter:')

# Create range filters for other columns
win_rate_filter = widgets.FloatRangeSlider(
    value=[0, 100], min=0, max=100, step=1, description='Win Rate Filter:', continuous_update=False
)
place_rate_filter = widgets.FloatRangeSlider(
    value=[0, 100], min=0, max=100, step=1, description='Place Rate Filter:', continuous_update=False
)
show_rate_filter = widgets.FloatRangeSlider(
    value=[0, 100], min=0, max=100, step=1, description='Show Rate Filter:', continuous_update=False
)
total_races_filter = widgets.IntRangeSlider(
    value=[df['Total Races'].min(), df['Total Races'].max()], min=df['Total Races'].min(), max=df['Total Races'].max(),
    step=1, description='Total Races Filter:', continuous_update=False
)
winnings_filter = widgets.FloatRangeSlider(
    value=[df['Total Winnings (ETH)'].min(), df['Total Winnings (ETH)'].max()], min=df['Total Winnings (ETH)'].min(),
    max=df['Total Winnings (ETH)'].max(), step=0.001, description='Winnings Filter:', continuous_update=False
)
purchase_price_filter = widgets.FloatRangeSlider(
    value=[df['Purchase Price (ETH)'].min(), df['Purchase Price (ETH)'].max()],
    min=df['Purchase Price (ETH)'].min(), max=df['Purchase Price (ETH)'].max(), step=0.001,
    description='Purchase Price Filter:', continuous_update=False
)

# Set the layout options for the widgets
widget_width = '250px'
widget_margin = '10px'

bloodline_filter.layout.width = widget_width
bloodline_filter.layout.margin = widget_margin

breed_filter.layout.width = widget_width
breed_filter.layout.margin = widget_margin

coat_color_filter.layout.width = widget_width
coat_color_filter.layout.margin = widget_margin

win_rate_filter.layout.width = widget_width
win_rate_filter.layout.margin = widget_margin

place_rate_filter.layout.width = widget_width
place_rate_filter.layout.margin = widget_margin

show_rate_filter.layout.width = widget_width
show_rate_filter.layout.margin = widget_margin

total_races_filter.layout.width = widget_width
total_races_filter.layout.margin = widget_margin

winnings_filter.layout.width = widget_width
winnings_filter.layout.margin = widget_margin

purchase_price_filter.layout.width = widget_width
purchase_price_filter.layout.margin = widget_margin

# Create a function to update the displayed data based on the selected filters
def update_data(
        bloodline_filter, breed_filter, coat_color_filter, win_rate_filter, place_rate_filter,
        show_rate_filter, total_races_filter, winnings_filter, purchase_price_filter
):
    bloodline_condition = True if bloodline_filter == 'All' else (df['Bloodline'] == bloodline_filter)
    breed_condition = True if breed_filter == 'All' else (df['Breed'] == breed_filter)
    coat_color_condition = True if coat_color_filter == 'All' else (df['Coat Color'] == coat_color_filter)

    # Apply the filters to the DataFrame
    filtered_data = df[
        bloodline_condition &
        breed_condition &
        coat_color_condition &
        (df['Free Win Rate'].str.rstrip('%').astype(float) >= win_rate_filter[0]) &
        (df['Free Win Rate'].str.rstrip('%').astype(float) <= win_rate_filter[1]) &
        (df['Overall Place Rate'].str.rstrip('%').astype(float) >= place_rate_filter[0]) &
        (df['Overall Place Rate'].str.rstrip('%').astype(float) <= place_rate_filter[1]) &
        (df['Overall Show Rate'].str.rstrip('%').astype(float) >= show_rate_filter[0]) &
        (df['Overall Show Rate'].str.rstrip('%').astype(float) <= show_rate_filter[1]) &
        (df['Total Races'] >= total_races_filter[0]) &
        (df['Total Races'] <= total_races_filter[1]) &
        (df['Total Winnings (ETH)'] >= winnings_filter[0]) &
        (df['Total Winnings (ETH)'] <= winnings_filter[1]) &
        (df['Purchase Price (ETH)'] >= purchase_price_filter[0]) &
        (df['Purchase Price (ETH)'] <= purchase_price_filter[1])
    ]

    # Update the summary statistics DataFrame
    summary_stats['Free Win Rate'] = filtered_data['Free Win Rate'].str.rstrip('%').astype(float)
    summary_stats['Overall Show Rate'] = filtered_data['Overall Show Rate'].str.rstrip('%').astype(float)

    display(filtered_data)


# Create an interactive display using the filters and update function
interactive_display = widgets.interactive_output(
    update_data,
    {
        'bloodline_filter': bloodline_filter,
        'breed_filter': breed_filter,
        'coat_color_filter': coat_color_filter,
        'win_rate_filter': win_rate_filter,
        'place_rate_filter': place_rate_filter,
        'show_rate_filter': show_rate_filter,
        'total_races_filter': total_races_filter,
        'winnings_filter': winnings_filter,
        'purchase_price_filter': purchase_price_filter
    }
)

# Create a grid layout for the filters
grid = widgets.GridBox(
    children=[
        bloodline_filter, breed_filter, coat_color_filter,
        win_rate_filter, place_rate_filter, show_rate_filter,
        total_races_filter, winnings_filter, purchase_price_filter
    ],
    layout=widgets.Layout(
        grid_template_columns='repeat(3, 1fr)',
        grid_gap='10px'
    )
)

summary_stats = df.loc[:, 'Free Win Rate':'Purchase Price (ETH)']

summary_stats['Free Win Rate'] = summary_stats['Free Win Rate'].str.rstrip('%').astype(float)
summary_stats['Overall Show Rate'] = summary_stats['Overall Show Rate'].str.rstrip('%').astype(float)

# Create a widget to display the summary statistics
summary_widget = widgets.Output()

# Define a function to update the summary widget
def update_summary():
    with summary_widget:
        summary_widget.clear_output()
        display(summary_stats.describe())

# Create a function to handle filter changes and trigger the update of the summary
def handle_filter_change(change):
    update_summary()

# Observe changes in the filter values and call the handle_filter_change function
bloodline_filter.observe(handle_filter_change, names='value')
breed_filter.observe(handle_filter_change, names='value')
coat_color_filter.observe(handle_filter_change, names='value')
win_rate_filter.observe(handle_filter_change, names='value')
place_rate_filter.observe(handle_filter_change, names='value')
show_rate_filter.observe(handle_filter_change, names='value')
total_races_filter.observe(handle_filter_change, names='value')
winnings_filter.observe(handle_filter_change, names='value')
purchase_price_filter.observe(handle_filter_change, names='value')

# Call the update_summary function initially to display the summary statistics
update_summary()

# Display the grid layout and the updated data
display(grid, interactive_display)
# Display the summary widget
display(summary_widget)

GridBox(children=(Dropdown(description='Bloodline Filter:', layout=Layout(margin='10px', width='250px'), optio…

Output()

Output()